In [5]:
import os
os.environ['PYTHONHASHSEED'] = '0'
os.environ['CUDA_LAUNCH_BLOCKING']='1'
import sys

sys.path.append('../TrainedNarrators/')
from data_utils import *
from model_utils import setupTokenizer
learning_rate = 3e-4
batch_size = 16
epochs = 20
warmup_steps = 0
modeltype = 'earlyfusion'
modelbase='facebook/bart-base'


In [6]:
import pickle as pk
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
tokenizer = tokenizer_ = setupTokenizer(modelbase=modelbase)
device = torch.device("cuda")

processed = pk.load(open('../dataset/train_dataset_org.dat', 'rb'))
dataset = RDFDataSetForTableStructured(tokenizer_,  processed, modelbase,max_preamble_len=160,
                                       max_len_trg=180, max_rate_toks=8,
                                       lower_narrations=True, process_target=True)

In [7]:
if 'bart' in modelbase:
    from narrations_models import BartNarrationModel
    model_generator = BartNarrationModel(
    vocab_size=len(tokenizer), model_type=modeltype,modelbase=modelbase)
else:
    from narrations_models import T5NarrationModel
    model_generator = T5NarrationModel(
    vocab_size=len(tokenizer), model_type=modeltype,modelbase=modelbase)

Model Type: earlyfusion


Some weights of DataNarrationBart were not initialized from the model checkpoint at facebook/bart-base and are newly initialized: ['encoder.relative_attention_for_table.query.bias', 'encoder.relative_attention_for_table.key.weight', 'encoder.relative_attention_for_table.query.weight', 'encoder.relative_attention_for_table.Er', 'encoder.relative_attention_for_table.value.bias', 'encoder.relative_attention_for_table.mask', 'encoder.relative_attention_for_table.value.weight', 'encoder.relative_attention_for_table.key.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
model_generator.generator.get_encoder().embed_tokens.weight.shape

torch.Size([50303, 768])

In [15]:
model_generator.aux_encoder

CollapsedMetricsTableEncoderBart(
  (token_embedding_layer): Embedding(50265, 768, padding_idx=1)
  (embedding_layer): Embedding(50265, 768, padding_idx=1)
  (metric_name_sa): SelfAttentionEncoderBart(
    (att_block): BartEncoderLayer(
      (self_attn): BartAttention(
        (k_proj): Linear(in_features=768, out_features=768, bias=True)
        (v_proj): Linear(in_features=768, out_features=768, bias=True)
        (q_proj): Linear(in_features=768, out_features=768, bias=True)
        (out_proj): Linear(in_features=768, out_features=768, bias=True)
      )
      (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (fc1): Linear(in_features=768, out_features=3072, bias=True)
      (fc2): Linear(in_features=3072, out_features=768, bias=True)
      (final_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    )
  )
  (metric_value_sa): SelfAttentionEncoderBart(
    (att_block): BartEncoderLayer(
      (self_attn): BartAttention(
        (k_

In [8]:
model_generator.generator.num_parameters()

141286912

In [ ]:
../TrainedNarrators/P-NarrationsModels/baselineoutputs_full_rated/

In [4]:
#model_generator.bartconfig

In [4]:
hh= modelbase.split('/')[1] if 'bart' in modelbase else modelbase

In [5]:
output_path='outputs_full_rated'
output_path = '../TrainedNarrators/P-NarrationsModels/' + \
    modeltype+output_path+'/'+hh+'/'

In [6]:
output_path

'../TrainedNarrators/P-NarrationsModels/baselineoutputs_full_rated/bart-large/'

In [7]:
model_generator.loadModel(model_path=output_path+'trained_model.pt')

In [8]:
#seed_everything(43)
def generateForSampleTopK(prompt,bs=4,use_top_k=False):
    #seed_everything(43)
    model_generator.generator.eval()
    if model_generator.aux_encoder is not None:
        model_generator.aux_encoder.eval()
    batch = dataset.processTableInfo(prompt)
    # batch=dataset.processTableInfo(test_sample[tidx])
    clb, di = batch['class_labels'].unsqueeze(0).to(
                device), batch['data_info'].unsqueeze(0).to(device)
    met, rate, val = batch['metrics_seq'].unsqueeze(0).to(device), batch['rates'].unsqueeze(
                0).to(device), batch['values'].unsqueeze(0).to(device)
    preamble_tokens = batch['preamble_tokens'].unsqueeze(
                0).to(device)
    preamble_attention_mask = batch['preamble_attention_mask'].unsqueeze(
                0).to(device)
    met_att = batch['metrics_attention'].unsqueeze(0).to(device)
    rate_att = batch['rate_attention'].unsqueeze(0).to(device)
    val_att = batch['value_attention'].unsqueeze(0).to(device)
    
    
    if model_generator.aux_encoder is not None:
        table_rep = model_generator.performAuxEncoding(
                    [met, met_att], [val, val_att], [rate, rate_att])
        if use_top_k:
            sample_outputs = model_generator.generator.generate(input_ids=preamble_tokens,
                                                                    attention_mask=preamble_attention_mask,
                                                                table_inputs=table_rep,
                                                                    table_attention_mask=None,
                                                                    do_sample=True, top_k=50, 
    top_p=0.95,
                                                                    max_length = 190,
                                                                early_stopping=True,
                                                                    use_cache=True,
                                                                     
                                                                    num_return_sequences=15,
                                                                    # bos_token_id=random.randint(1,30000),
                                                                    )
        else:
            sample_outputs = model_generator.generator.generate(input_ids=preamble_tokens,
                                                                    attention_mask=preamble_attention_mask,
                                                                    table_inputs=table_rep,
                                                                    table_attention_mask=None,
                                                                    num_beams=bs,
                                                                    repetition_penalty=1.5,
                                                                    length_penalty=2.6,
                                                                    early_stopping=True,
                                                                    use_cache=True,
                                                                    max_length=190,
                                                                    no_repeat_ngram_size=2,
                                                                    num_return_sequences=bs,
                                                                    do_sample=False
                                                                    # bos_token_id=random.randint(1,30000),
                                                                    )
        
    else:
        if use_top_k:
            sample_outputs = model_generator.generator.generate(input_ids=preamble_tokens,
                                                                    attention_mask=preamble_attention_mask,
                                                                    do_sample=True, top_k=50, 
    top_p=0.95,
                                                                    max_length = 190,
                                                                early_stopping=True,
                                                                    use_cache=True,
                                                                     
                                                                    num_return_sequences=15,
                                                                    # bos_token_id=random.randint(1,30000),
                                                                    )
        else:
            sample_outputs = model_generator.generator.generate(input_ids=preamble_tokens,
                                                                    attention_mask=preamble_attention_mask,
                                                                    num_beams=bs,
                                                                    repetition_penalty=1.5,
                                                                    length_penalty=8.6,
                                                                    early_stopping=True,
                                                                    use_cache=True,
                                                                    max_length=190,
                                                                    no_repeat_ngram_size=2,
                                                                    num_return_sequences=bs,
                                                                    do_sample=False
                                                                    # bos_token_id=random.randint(1,30000),
                                                                    )
    ss = [tokenizer.decode(s,
                                  skip_special_tokens=True,
                                  clean_up_tokenization_spaces=True) for s in sample_outputs]
    return ss
        

In [10]:
test_data_u = json.load(open('../dataset/test set.json'))
test_sample_u = []
eval_tables_u = []
for idx, pc in enumerate(test_data_u):
    #print(idx)
    test_sample_u.append(processInputTableAndNarrations(
        pc, identical_metrics=identicals,augnment_metrics=True))
    


In [11]:
len(test_data_u)

100

In [12]:
tidx=0
pc = test_data_u[tidx]
prep=processInputTableAndNarrations(
        pc, identical_metrics=identicals,augnment_metrics=True)
prep

{'preamble': '<MetricsInfo> precision | VALUE_HIGH | 91.3% <|> f1score | VALUE_HIGH | 88.89% <|> accuracy | VALUE_HIGH | 90.67% <|> sensitivity | VALUE_HIGH | 87.29% && sensitivity | also_known_as | recall  <|section-sep|> <TaskDec> ml_task | dataset_attributes | <|IMBALANCED|> && ml_task | class_labels | #CA and #CB  <|section-sep|> <|table2text|> ',
 'classes': ['#CA', '#CB'],
 'dataset_attribute': ['<|IMBALANCED|>'],
 'metrics': ['sensitivity', 'accuracy', 'f1score', 'precision'],
 'values': ['87.29%', '90.67%', '88.89%', '91.3%'],
 'rates': ['HIGH', 'HIGH', 'HIGH', 'HIGH'],
 'narration': "The classifier was able to produce fairly high scores across the metrics sensitivity, accuracy, precision and F1score. Specifically, for the sensitivity it scored 87.29%, accuracy (96.67%) and precision (91.3%) with the F1score equal to 88.89%. These scores suggest that the model will incorrectly assign the wrong labels for only a small number of test cases. Overall, the model's prediction decisio

In [13]:
tidx=0
pc = test_data_u[tidx]
prep=processInputTableAndNarrations(
        pc, identical_metrics=identicals,augnment_metrics=True)
print(prep)
outp=generateForSampleTopK(prep,bs=11,use_top_k=False)

{'preamble': '<MetricsInfo> precision | VALUE_HIGH | 91.3% <|> sensitivity | VALUE_HIGH | 87.29% && sensitivity | also_known_as | recall <|> accuracy | VALUE_HIGH | 90.67% <|> f1score | VALUE_HIGH | 88.89%  <|section-sep|> <TaskDec> ml_task | dataset_attributes | <|IMBALANCED|> && ml_task | class_labels | #CA and #CB  <|section-sep|> <|table2text|> ', 'classes': ['#CA', '#CB'], 'dataset_attribute': ['<|IMBALANCED|>'], 'metrics': ['f1score', 'accuracy', 'precision', 'sensitivity'], 'values': ['88.89%', '90.67%', '91.3%', '87.29%'], 'rates': ['HIGH', 'HIGH', 'HIGH', 'HIGH'], 'narration': "The classifier was able to produce fairly high scores across the metrics sensitivity, accuracy, precision and F1score. Specifically, for the sensitivity it scored 87.29%, accuracy (96.67%) and precision (91.3%) with the F1score equal to 88.89%. These scores suggest that the model will incorrectly assign the wrong labels for only a small number of test cases. Overall, the model's prediction decisions are

In [14]:
outp

["Trained to tell-apart the examples belonging to the class labels #CA and #CB, the model's classification prowess is characterized by the scores 86.49%, 75.29%, 77.04%, and 81.61%, respectively, across the metrics sensitivity, precision, AUC, and accuracy. From these scores, we can conclude that the learning algorithm employed to solve this ML task has a moderately high prediction performance and will be able to correctly classify most test samples with only a few instances misclassified.",
 "Trained to tell-apart the examples belonging to the class labels #CA and #CB, the model's classification prowess is characterized by the scores 86.49%, 75.29%, 77.04%, and 81.61%, respectively, across the metrics sensitivity, precision, AUC, and accuracy. From these scores, we can conclude that the learning algorithm employed to solve this ML task has a moderately high prediction performance and will be able to correctly classify most test samples with only a small margin of error.",
 "Trained to

In [20]:
ttt= T5Tokenizer.from_pretrained('t5-base')

In [22]:
#print(ttt.get_vocab())

In [37]:
sentences=[pc['narration'] for pc in full_data]
sentences[1],ttt.encode('<acc_diff>')

('To summarise, this is a good performing model with high accuracy of 92.78%, although a recall of 81.15% means that the model misclassified <rec_diff> of the majority class C1 as C2. The model performs well, with high accuracy (92.78%) and very high auc (96.38%) and very high precision (98.02). A precision score this high means that 98.02% of identifications predicited as class C1 were actually C1.',
 [3, 2, 6004, 834, 26, 5982, 3155, 1])

In [30]:
sentences = ''.join(sentences)

In [31]:
sentences[:20]

'T h i s c l a s s i '

In [23]:
#print(tokenizer.get_vocab())

In [32]:
table_ref_metrics = [[m.lower().strip() for m in test_sample_u[idx]['metrics']] for idx in range(len(test_sample_u))]

In [33]:
table_ref_metrics 

[['specificity', 'accuracy', 'f2score', 'sensitivity', 'precision']]

In [7]:
table_ref_metrics = [[m.lower().strip() for m in test_sample[idx]['metrics']] for idx in range(len(test_sample))]

# replace metrics such as sensitivity with recall
cleans_table_refs =[]
for r in copy.deepcopy( table_ref_metrics):
    if 'sensitivity' in set(r):
        idx = r.index('sensitivity')
        r[idx] ='recall'
    cleans_table_refs.append(r)

In [8]:
tidx=4
test_sample[tidx]

{'preamble': '<MetricsInfo> f2score | VALUE_HIGH | 84.33% <|> accuracy | VALUE_HIGH | 86.11% <|> precision | VALUE_HIGH | 89.07% <|> auc | VALUE_HIGH | 90.09% <|> sensitivity | VALUE_HIGH | 84.29% && sensitivity | also_known_as | recall  <|section-sep|> <TaskDec> ml_task | dataset_attributes | <|BALANCED|> && ml_task | class_labels | #CA and #CB  <|section-sep|> <|table2text|> ',
 'classes': ['#CA', '#CB'],
 'dataset_attribute': ['<|BALANCED|>'],
 'metrics': ['sensitivity', 'auc', 'accuracy', 'precision', 'f2score'],
 'values': ['84.29%', '90.09%', '86.11%', '89.07%', '84.33%'],
 'rates': ['HIGH', 'HIGH', 'HIGH', 'HIGH', 'HIGH'],
 'narration': "Sensitivity, accuracy, AUC and precision scores of 84.29%, 86.11%, and 89.07% respectively show or indicate how good the model's performance is on this binary classification task. From the F2score, precision and sensitivity scores, we can see that the false positive rate is very low. In summary, only a small number of test cases are likely to be

In [9]:
outp=generateForSampleTopK(test_sample[tidx],use_top_k=True)

Global seed set to 43


In [24]:
def extractMetricMentions(narration,replace_identical=True,ref_mentions=None):
    metrics_list = ['recall', 'sensitivity', 'f1-score',
                    'precision', 'f2-score', 'f1','f1score','f2score','g-mean','gmean',
                    'f2', 'accuracy',
                    'auc',
                    'specificity', ]
    nn = normalize_text(narration).split()
    # replace identical metrics such as recall and sensitivity based on the mention in the table
    
    # Take care of when recall and sensitivity is mentioned together
    
    found_metrics = [s.lower()#.replace('-score', '').replace('score', '')
                     for s in set(metrics_list).intersection(nn)]
    if replace_identical:
        if 'recall' in set(found_metrics) and 'sensitivity' in set(found_metrics):
            if 'sensitivity' in ref_mentions:
                found_metrics = ' '.join(found_metrics).replace('recall','').split()
            elif 'sensitivity' in ref_mentions:
                found_metrics = ' '.join(found_metrics).replace('sensitivity','').split()
    return found_metrics
def metricMentionScore(reference_metrics,sys_output):
    ref_counts =  np.sum([len(e) for e in reference_metrics])
    sys_mentions = [extractMetricMentions(r,ref_mentions=t) for r,t in zip(sys_output,reference_metrics)]
    g_sysCount = np.sum([len(e) for e in sys_mentions])
    overlap=np.sum([getOverlap(r,b) for r,b in zip(reference_metrics,sys_mentions)])
    recall=overlap/ref_counts
    precision=overlap/g_sysCount
    f1= 2*(recall*precision)/(recall+precision)
    return dict(f1_score=round(f1,5),recall=round(recall,5),precision=round(precision,5))#,sys_mentions

In [25]:
def classLabelsEval(reference_classes,generated,penalty =-1):
    # In some cases the model will generate the phrase "binary classification" or "multi-class"
    # "2 classes", "3 classes","two labels", "two classes"
    false_mentions = 1
    if any(ele in generated for ele in ['binary','two classes','two class labels','two-labels','2 classes',
                                        'two-classes','two class labels','2 labels','2 classes']) and len(reference_classes)> 2:
        # should have generated multi-class
        false_mentions = penalty
        
    if any(ele in generated for ele in ['multi-class', 'multi class', 'multi classes','multi labels',
                                        'three-classes','three classes',
                                        'three class labels',
                                        '3-labels',
                                        '3-classes',
                                       'four classes',
                                        'four class labels',
                                        '4 labels',
                                        '4 classes']) and len(reference_classes)< 3:
        # should have generated binary
        false_mentions = penalty
        
    
    classes_tokens = [f'c{i}' for i in range(1, 6)]
    nn= normalize_text(generated).split()
    found_classes = [s.lower() for s in set(classes_tokens).intersection(nn)]
    n_found = 0
    label_mentions ={}
    label_was_mentioned=False
    predicted_text = normalize_text(generated)
    for cl in classes_tokens:
        c= predicted_text.count(cl)
        if cl in reference_classes:
            # The class label exists in the reference class list so the model gets a score of 1
            label_mentions[cl] = 1
        else:
            if c >0:
                # Invalid class label was mentioned
                # A penalty is assigned t
                label_mentions[cl] = penalty
            else:
                # No mention of the class 
                label_mentions[cl] = 0
        if c > 0:
            label_was_mentioned=True
    
    if not label_was_mentioned:
        # if no label was mentioned, we assume  they were mentioned implicitly
        return 1
    else:
        # Check to make sure only the right labels were accounted for
        score = np.mean([v for v in list(label_mentions.values()) if v !=0]+[false_mentions])
        return np.round(score,2)

In [26]:
def checkForDataDistributionMentions(text,dataset_info):
    balanced_list = ['balanced', 'balance','identical', 'similar distribution']
    imblanced_list = ['disproportionate','imbalance','class imbalance','imbalanced']
    if any(ele in text.split() for ele in balanced_list):
        return 'balanced'
    if any(ele in text for ele in imblanced_list):
        return 'imbalanced'

In [27]:
outp=[o.replace('G-Mean','GMean') for o in outp]

In [38]:
table_ref_metrics

[['specificity', 'accuracy', 'f2score', 'sensitivity', 'precision']]

In [40]:
extractMetricMentions(normalize_text(outp[0]),ref_mentions=table_ref_metrics[0])

['f1score', 'precision', 'f2score', 'accuracy', 'specificity', 'sensitivity']

In [53]:
checkForDataDistributionMentions(outp[-1],'balanced')

'imbalanced'

In [35]:
normalize_text(outp[-1])

'the following are the scores achieved by the classifier on this binary classification task : ( 1 ) accuracy equal to 86.11 % with the auc , ( 2 ) sensitivity and ( 3 ) precision equal to 90.09 % and ( 4 ) f1score of 84.33 % . in addition , the prediction performance of the model is high despite the class imbalance . judging based on the scores , it would be safe to conclude that this model has a moderate performance as it is likely to misclassify some test examples especially those drawn randomly from the class label .'

In [41]:
for s in outp:
    print(metricMentionScore([table_ref_metrics[tidx]],[s]))

{'f1_score': 0.90909, 'recall': 1.0, 'precision': 0.83333}
{'f1_score': 0.90909, 'recall': 1.0, 'precision': 0.83333}
{'f1_score': 0.66667, 'recall': 0.6, 'precision': 0.75}
{'f1_score': 0.66667, 'recall': 0.6, 'precision': 0.75}
{'f1_score': 0.90909, 'recall': 1.0, 'precision': 0.83333}
{'f1_score': 0.90909, 'recall': 1.0, 'precision': 0.83333}
{'f1_score': 0.66667, 'recall': 0.6, 'precision': 0.75}
{'f1_score': 0.66667, 'recall': 0.6, 'precision': 0.75}
